In [2]:
#instalação de pacotes necessários
#install.packages("e1071")
#install.packages("caret")
#install.packages("mlbench")
#install.packages("mice")
library(caret)
library(mlbench)
library(mice)

In [4]:
#leitura dos dados

dados <- read.csv("databases/4 – Banco - Dados.csv")

head(dados)

,age,job,marital,education,default,balance,housing,loan,y
,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>
1,30,unemployed,married,primary,no,1787,no,no,no
2,33,services,married,secondary,no,4789,yes,yes,no
3,35,management,single,tertiary,no,1350,yes,no,no
4,30,management,married,tertiary,no,1476,yes,yes,no
5,59,bluecollar,married,secondary,no,0,yes,no,no
6,35,management,single,tertiary,no,747,no,no,no


In [5]:
#separando em bases de teste e treinamento
set.seed(2034)
indices <-  createDataPartition(dados$y, p = 0.80, list = FALSE)

treino <- dados[indices, ]
teste <- dados[-indices, ]

In [6]:
#treinando o modelo com holdout

set.seed(2034)
rna <- train(y ~ ., data = treino, method = "nnet", trace = FALSE)
rna

Neural Network 

241 samples
  8 predictor
  2 classes: 'no', 'yes' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 241, 241, 241, 241, 241, 241, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa        
  1     0e+00  0.8750789   0.0000000000
  1     1e-04  0.8750789   0.0000000000
  1     1e-01  0.8655094   0.0231396199
  3     0e+00  0.8705219  -0.0005433962
  3     1e-04  0.8676178   0.0118364838
  3     1e-01  0.8497132   0.0518217575
  5     0e+00  0.8750789   0.0000000000
  5     1e-04  0.8651299   0.0003691639
  5     1e-01  0.8471367   0.0837014804

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were size = 1 and decay = 1e-04.

In [7]:
#predicoes dos valores do conjunto de teste
predict_rna <- predict(rna, teste)

#matriz de confusao
confusionMatrix(predict_rna, as.factor(teste$y))

Confusion Matrix and Statistics

          Reference
Prediction no yes
       no  52   7
       yes  0   0
                                          
               Accuracy : 0.8814          
                 95% CI : (0.7707, 0.9509)
    No Information Rate : 0.8814          
    P-Value [Acc > NIR] : 0.59891         
                                          
                  Kappa : 0               
                                          
 Mcnemar's Test P-Value : 0.02334         
                                          
            Sensitivity : 1.0000          
            Specificity : 0.0000          
         Pos Pred Value : 0.8814          
         Neg Pred Value :    NaN          
             Prevalence : 0.8814          
         Detection Rate : 0.8814          
   Detection Prevalence : 1.0000          
      Balanced Accuracy : 0.5000          
                                          
       'Positive' Class : no              
                                 

In [11]:
#usando cross validation
ctrl <- trainControl(method = "cv", number = 10)

#executando rna com control
set.seed(2034)
rna <- train(y ~ ., data = treino, method = "nnet", trace = FALSE, trControl = ctrl)
predict_rna <- predict(rna, teste)

confusionMatrix(predict_rna, as.factor(teste$y))

Confusion Matrix and Statistics

          Reference
Prediction no yes
       no  52   7
       yes  0   0
                                          
               Accuracy : 0.8814          
                 95% CI : (0.7707, 0.9509)
    No Information Rate : 0.8814          
    P-Value [Acc > NIR] : 0.59891         
                                          
                  Kappa : 0               
                                          
 Mcnemar's Test P-Value : 0.02334         
                                          
            Sensitivity : 1.0000          
            Specificity : 0.0000          
         Pos Pred Value : 0.8814          
         Neg Pred Value :    NaN          
             Prevalence : 0.8814          
         Detection Rate : 0.8814          
   Detection Prevalence : 1.0000          
      Balanced Accuracy : 0.5000          
                                          
       'Positive' Class : no              
                                 

In [12]:
#parametrização
grid <- expand.grid(size = seq(from = 1, to = 35, by = 10), decay = seq(from = 0.1, to = 0.6, by = 0.3))

set.seed(2034)
rna <- train(form = y ~ ., data = treino, method ="nnet", tuneGrid = grid, trControl = ctrl, maxit = 2000, trace = FALSE)

In [13]:
rna

Neural Network 

241 samples
  8 predictor
  2 classes: 'no', 'yes' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 217, 216, 217, 217, 217, 217, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa      
   1    0.1    0.8590000  -0.01777778
   1    0.4    0.8715000   0.00000000
  11    0.1    0.8591667   0.13760684
  11    0.4    0.8715000   0.00000000
  21    0.1    0.8505000   0.14637363
  21    0.4    0.8715000   0.00000000
  31    0.1    0.8550000   0.16610743
  31    0.4    0.8715000   0.00000000

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were size = 1 and decay = 0.4.

In [14]:
#faz a predicao na base de teste e mostra a matriz de confusao
predict_rna <- predict(rna, teste)
confusionMatrix(predict_rna, as.factor(teste$y))


Confusion Matrix and Statistics

          Reference
Prediction no yes
       no  52   7
       yes  0   0
                                          
               Accuracy : 0.8814          
                 95% CI : (0.7707, 0.9509)
    No Information Rate : 0.8814          
    P-Value [Acc > NIR] : 0.59891         
                                          
                  Kappa : 0               
                                          
 Mcnemar's Test P-Value : 0.02334         
                                          
            Sensitivity : 1.0000          
            Specificity : 0.0000          
         Pos Pred Value : 0.8814          
         Neg Pred Value :    NaN          
             Prevalence : 0.8814          
         Detection Rate : 0.8814          
   Detection Prevalence : 1.0000          
      Balanced Accuracy : 0.5000          
                                          
       'Positive' Class : no              
                                 

In [16]:
#predicao para novos casos

dados_novos_casos <- read.csv("databases/4 - Banco - Novos Casos.csv")
View(dados_novos_casos)

predict_rna <- predict(rna, dados_novos_casos)
dados_novos_casos$y <- NULL
resultado <- cbind(dados_novos_casos, predict_rna)
View(resultado)

age,job,marital,education,default,balance,housing,loan,y
<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>
60,unemployed,married,primary,no,2000,yes,yes,?
33,services,married,secondary,yes,3000,yes,no,?
15,management,single,tertiary,no,1350,yes,no,?


age,job,marital,education,default,balance,housing,loan,predict_rna
<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<fct>
60,unemployed,married,primary,no,2000,yes,yes,no
33,services,married,secondary,yes,3000,yes,no,no
15,management,single,tertiary,no,1350,yes,no,no
